<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_TR1_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 1 - SQL

Nome: Carlos Henrique Valério de Moraes

Matrícula: (professor)

# Banco Original em Arquivo CSV

## Analisando Arquivo CSV

In [ ]:
import pandas as pd

banco = pd.read_csv('https://github.com/valerio-unifei/ECAA07/raw/main/Bancos/ecaa07inv_079.csv')
banco.head()

,cnpj,id,classe,fundo,minimo,taxa,estudo,fator,rendimento,data,patrimonio,calculo,momento,custo
0,00.066.670/0001-00,41562385000168,Fundo de Renda Fixa,SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVEST...,0.0,55,41,20.006406,1.000409,20210722,20008178.54,0.323914,-18,430.591302
1,00.066.670/0001-00,41562385000168,Fundo de Renda Fixa,SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVEST...,0.0,55,41,20.006406,0.997899,20210723,19957973.10,0.253912,-9,39.426419
2,00.066.670/0001-00,41562385000168,Fundo de Renda Fixa,SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVEST...,0.0,55,41,20.006406,0.996144,20210726,19922874.43,0.290206,15,123.604320
3,00.066.670/0001-00,41562385000168,Fundo de Renda Fixa,SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVEST...,0.0,55,41,20.006406,0.993561,20210727,19871229.90,0.500330,22,138.320207
4,00.066.670/0001-00,41562385000168,Fundo de Renda Fixa,SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVEST...,0.0,55,41,20.006406,0.993271,20210728,19865417.89,0.110878,-29,156.353707


In [ ]:
#convertendo campo numérico para data
banco['data'] = pd.to_datetime(banco['data'], format='%Y%m%d')
# ordenando por id do fundo e data do rendimento
banco = banco.sort_values(by = ['id','data'])
banco.head()

,cnpj,id,classe,fundo,minimo,taxa,estudo,fator,rendimento,data,patrimonio,calculo,momento,custo
200,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTI...,0.0,7,14,36.949901,237.685355,2021-07-22,20613050.59,0.517853,-20,87.546986
201,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTI...,0.0,7,14,36.949901,237.408022,2021-07-23,20588999.16,0.189034,-26,57.316826
202,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTI...,0.0,7,14,36.949901,237.029262,2021-07-26,20556151.57,0.382121,-12,50.616987
203,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTI...,0.0,7,14,36.949901,235.561754,2021-07-27,20428883.21,0.896952,7,58.453555
204,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTI...,0.0,7,14,36.949901,236.282369,2021-07-28,20491377.95,0.988759,11,266.150421


In [ ]:
# informações sobre a base
print(banco.info())
print(banco.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 200 to 49
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   cnpj        250 non-null    object        
 1   id          250 non-null    int64         
 2   classe      250 non-null    object        
 3   fundo       250 non-null    object        
 4   minimo      250 non-null    float64       
 5   taxa        250 non-null    int64         
 6   estudo      250 non-null    int64         
 7   fator       250 non-null    float64       
 8   rendimento  250 non-null    float64       
 9   data        250 non-null    datetime64[ns]
 10  patrimonio  250 non-null    float64       
 11  calculo     250 non-null    float64       
 12  momento     250 non-null    int64         
 13  custo       250 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(3)
memory usage: 29.3+ KB
None
cnpj            4
id              5
cla

## Criando Arquivo SQLite com Tabela Auxiliar

In [ ]:
!rm ecaa07inv_079.db

rm: cannot remove 'ecaa07inv_079.db': No such file or directory


In [ ]:
import sqlite3
with sqlite3.connect('ecaa07inv_079.db') as conn:
  banco.to_sql('auxiliar',conn)

# Banco de Dados em SQLite

In [ ]:
%load_ext sql

%sql sqlite:///ecaa07inv_079.db

'Connected: @ecaa07inv_079.db'

## Criando Tabelas do Banco

In [ ]:
%%sql 
CREATE TABLE IF NOT EXISTS classes (
  classe_id   INTEGER PRIMARY KEY AUTOINCREMENT,
  classe      TEXT NOT NULL
);

 * sqlite:///ecaa07inv_079.db
Done.


[]

In [ ]:
%%sql 
CREATE TABLE IF NOT EXISTS fundos (
  fundo_id    INTEGER PRIMARY KEY AUTOINCREMENT,
  cnpj        TEXT,
  id          INTEGER NOT NULL,
  fundo       TEXT NOT NULL,
  classe_id   INTEGER REFERENCES classes(classe_id) ON DELETE CASCADE,
  minimo      REAL,
  taxa        INTEGER,
  estudo      INTEGER,
  fator       REAL
);

 * sqlite:///ecaa07inv_079.db
Done.


[]

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS rendimentos (
  rendimento_id INTEGER PRIMARY KEY AUTOINCREMENT,
  fundo_id      INTEGER REFERENCES fundos(fundo_id) ON DELETE CASCADE,
  rendimento    REAL,
  data          DATETIME,
  patrimonio    REAL,
  calculo       REAL,
  momento       INTEGER,
  custo         REAL
);

 * sqlite:///ecaa07inv_079.db
Done.


[]

### Verificando estrutura

In [ ]:
%sql SELECT * FROM sqlite_master

 * sqlite:///ecaa07inv_079.db
Done.


type,name,tbl_name,rootpage,sql
table,auxiliar,auxiliar,2,"CREATE TABLE ""auxiliar"" (""index"" INTEGER, ""cnpj"" TEXT, ""id"" INTEGER, ""classe"" TEXT, ""fundo"" TEXT, ""minimo"" REAL, ""taxa"" INTEGER, ""estudo"" INTEGER, ""fator"" REAL, ""rendimento"" REAL, ""data"" TIMESTAMP, ""patrimonio"" REAL, ""calculo"" REAL, ""momento"" INTEGER, ""custo"" REAL)"
index,ix_auxiliar_index,auxiliar,3,"CREATE INDEX ""ix_auxiliar_index""ON ""auxiliar"" (""index"")"
table,classes,classes,17,"CREATE TABLE classes ( classe_id INTEGER PRIMARY KEY AUTOINCREMENT, classe TEXT NOT NULL)"
table,sqlite_sequence,sqlite_sequence,18,"CREATE TABLE sqlite_sequence(name,seq)"
table,fundos,fundos,19,"CREATE TABLE fundos ( fundo_id INTEGER PRIMARY KEY AUTOINCREMENT, cnpj TEXT, id INTEGER NOT NULL, fundo TEXT NOT NULL, classe_id INTEGER REFERENCES classes(classe_id) ON DELETE CASCADE, minimo REAL, taxa INTEGER, estudo INTEGER, fator REAL)"
table,rendimentos,rendimentos,20,"CREATE TABLE rendimentos ( rendimento_id INTEGER PRIMARY KEY AUTOINCREMENT, fundo_id INTEGER REFERENCES fundos(fundo_id) ON DELETE CASCADE, rendimento REAL, data DATETIME, patrimonio REAL, calculo REAL, momento INTEGER, custo REAL)"


# Inserindo Dados

## Usando Tabela Auxiliar na Importação

In [ ]:
%sql SELECT * FROM auxiliar LIMIT 10

 * sqlite:///ecaa07inv_079.db
Done.


index,cnpj,id,classe,fundo,minimo,taxa,estudo,fator,rendimento,data,patrimonio,calculo,momento,custo
200,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,237.6853546,2021-07-22 00:00:00,20613050.59,0.5178534419869525,-20,87.54698613446459
201,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,237.4080219,2021-07-23 00:00:00,20588999.16,0.18903449179521248,-26,57.316826184948596
202,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,237.02926230000003,2021-07-26 00:00:00,20556151.57,0.3821210008963122,-12,50.61698652191238
203,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,235.56175380000002,2021-07-27 00:00:00,20428883.21,0.8969518081763258,7,58.453555134679526
204,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,236.28236940000002,2021-07-28 00:00:00,20491377.95,0.9887593320035211,11,266.1504207783957
205,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,236.570662,2021-07-29 00:00:00,20516379.87,0.20445775165392788,4,80.8965307723602
206,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.28135419999998,2021-07-30 00:00:00,20317841.69,0.7487975100873017,-22,8.410159777971327
207,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.91014959999998,2021-08-02 00:00:00,20372373.41,0.319698180745692,-25,212.1943817274697
208,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.8713224,2021-08-03 00:00:00,20369006.16,0.4437179997111371,20,59.68849697278472
209,00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.2195759,2021-08-04 00:00:00,20312484.02,0.7548413386316962,-22,375.328010299221


In [ ]:
%sql INSERT INTO classes (classe) SELECT DISTINCT classe FROM auxiliar

 * sqlite:///ecaa07inv_079.db
3 rows affected.


[]

In [ ]:
%%sql
  INSERT INTO fundos (cnpj, id, fundo, minimo, taxa, estudo, fator, classe_id)
  SELECT DISTINCT a.cnpj, a.id, a.fundo, a.minimo, a.taxa, a.estudo, a.fator, b.classe_id
  FROM auxiliar as a INNER JOIN classes as b ON a.classe = b.classe

 * sqlite:///ecaa07inv_079.db
5 rows affected.


[]

In [ ]:
%%sql
  INSERT INTO rendimentos (fundo_id, rendimento, data, patrimonio, calculo, momento, custo)
  SELECT DISTINCT a.fundo_id, b.rendimento, b.data, b.patrimonio, b.calculo, b.momento, b.custo
  FROM fundos as a INNER JOIN auxiliar as b ON a.id = b.id

 * sqlite:///ecaa07inv_079.db
250 rows affected.


[]

In [ ]:
%sql DROP TABLE IF EXISTS auxiliar

 * sqlite:///ecaa07inv_079.db
Done.


[]

# Criando consultas

## Ajustando SQL

In [ ]:
%%sql 
  SELECT b.cnpj, b.id, a.classe, b.fundo, b.minimo, b.taxa, b.estudo, b.fator, c.rendimento, c.data, c.patrimonio, c.calculo, c.momento, c.custo
  FROM classes as a INNER JOIN fundos as b ON a.classe_id = b.classe_id
    INNER JOIN rendimentos as c ON b.fundo_id = c.fundo_id
  ORDER BY b.id, c.data
  LIMIT 10

 * sqlite:///ecaa07inv_079.db
Done.


cnpj,id,classe,fundo,minimo,taxa,estudo,fator,rendimento,data,patrimonio,calculo,momento,custo
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,237.6853546,2021-07-22 00:00:00,20613050.59,0.5178534419869525,-20,87.54698613446459
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,237.4080219,2021-07-23 00:00:00,20588999.16,0.18903449179521248,-26,57.316826184948596
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,237.02926230000003,2021-07-26 00:00:00,20556151.57,0.3821210008963122,-12,50.61698652191238
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,235.56175380000002,2021-07-27 00:00:00,20428883.21,0.8969518081763258,7,58.453555134679526
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,236.28236940000002,2021-07-28 00:00:00,20491377.95,0.9887593320035211,11,266.1504207783957
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,236.570662,2021-07-29 00:00:00,20516379.87,0.20445775165392788,4,80.8965307723602
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.28135419999998,2021-07-30 00:00:00,20317841.69,0.7487975100873017,-22,8.410159777971327
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.91014959999998,2021-08-02 00:00:00,20372373.41,0.319698180745692,-25,212.1943817274697
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.8713224,2021-08-03 00:00:00,20369006.16,0.4437179997111371,20,59.68849697278472
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,234.2195759,2021-08-04 00:00:00,20312484.02,0.7548413386316962,-22,375.328010299221


In [ ]:
%%sql 
SELECT b.fundo, (MAX(c.patrimonio) - MIN(c.patrimonio))/(MAX(JULIANDAY(c.data)) - MIN(JULIANDAY(c.data))) as diariomedio
    FROM fundos as b INNER JOIN rendimentos as c ON b.fundo_id = c.fundo_id
    GROUP BY b.fundo
    ORDER BY diariomedio DESC

 * sqlite:///ecaa07inv_079.db
Done.


fundo,diariomedio
FUNDO DE INVESTIMENTO EM AÇÕES CAIXA BDR NÍVEL I,861894.2447142856
ITAÚ FLEXPREV JURO REAL TARGET 2026 RENDA FIXA FUNDO DE INV EM COTAS DE FUNDOS DE INV,843450.2261428573
VTR DOUBLE INCOME FUNDO DE INVESTIMENTO RENDA FIXA,30574.72585714284
LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,11230.986571428552
SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVESTIMENTO EM INFRAESTRUTURA RENDA FIXA,8320.62357142855


## Criando VIEWS

In [ ]:
%%sql 
CREATE VIEW IF NOT EXISTS original as
    SELECT b.cnpj, b.id, a.classe, b.fundo, b.minimo, b.taxa, b.estudo, b.fator, c.rendimento, c.data, c.patrimonio, c.calculo, c.momento, c.custo
    FROM classes as a INNER JOIN fundos as b ON a.classe_id = b.classe_id
      INNER JOIN rendimentos as c ON b.fundo_id = c.fundo_id;

 * sqlite:///ecaa07inv_079.db
Done.


[]

In [ ]:
%%sql 
CREATE VIEW IF NOT EXISTS melhor_fundo as
  SELECT *

 * sqlite:///ecaa07inv_079.db
Done.


[]

In [ ]:
%%sql
  SELECT * from original

 * sqlite:///ecaa07inv_079.db
Done.


fundo,rend_real,dias,dmax,dmin,p_fim,p_ini
FUNDO DE INVESTIMENTO EM AÇÕES CAIXA BDR NÍVEL I,226692.0044285723,70.0,2021-09-30 00:00:00,2021-07-22 00:00:00,863989586.94,848121146.63
ITAÚ FLEXPREV JURO REAL TARGET 2026 RENDA FIXA FUNDO DE INV EM COTAS DE FUNDOS DE INV,-843450.2261428573,70.0,2021-09-30 00:00:00,2021-07-22 00:00:00,193080491.95,252122007.78
LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,-11063.989857142844,70.0,2021-09-30 00:00:00,2021-07-22 00:00:00,19838571.3,20613050.59
SÃO MIGUEL ARCANJO FUNDO INCENTIVADO DE INVESTIMENTO EM INFRAESTRUTURA RENDA FIXA,-2348.8618571428315,70.0,2021-09-30 00:00:00,2021-07-22 00:00:00,19843758.21,20008178.54
VTR DOUBLE INCOME FUNDO DE INVESTIMENTO RENDA FIXA,-30574.72585714284,70.0,2021-09-30 00:00:00,2021-07-22 00:00:00,21020183.76,23160414.57


### Testando Views

In [ ]:
%sql SELECT * FROM original

 * sqlite:///ecaa07inv_079.db
Done.


cnpj,id,classe,fundo,minimo,taxa,estudo,fator,rendimento,data,patrimonio,calculo,momento,custo
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,228.6201815,2021-09-28 00:00:00,19826881.53,0.4385695030499981,15,126.7406616771442
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,228.7008028,2021-09-29 00:00:00,19833873.34,0.022837320023968318,-22,68.09626327969065
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,228.754974098,2021-09-30 00:00:00,19838571.3,0.9544583447778432,18,69.72938590410038
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,228.76729139999998,2021-09-20 00:00:00,19839639.51,0.8539842564241644,19,231.10333158426002
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,229.18495040000002,2021-09-08 00:00:00,19875860.61,0.14672374193973847,10,336.09411493279913
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,229.37383050000003,2021-08-18 00:00:00,19892241.07,0.5738371916638056,10,312.4328310370134
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,229.5553941,2021-08-17 00:00:00,19907987.01,0.13580440526850268,-22,235.74805082719556
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,229.9868655,2021-09-21 00:00:00,19945405.99,0.2934589110539695,-19,154.95238044110388
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,230.1164151,2021-09-10 00:00:00,19956641.06,0.02270506401228212,7,442.02192986247735
00.066.670/0001-00,12518290000149,Fundo Multimercado,LIN FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR,0.0,7,14,36.94990082355038,230.35019920000002,2021-09-09 00:00:00,19976915.78,0.31667105139033197,-27,101.48352156751899


In [ ]:
%sql SELECT * FROM melhor_fundo

 * sqlite:///ecaa07inv_079.db
Done.


fundo,diariomedio
FUNDO DE INVESTIMENTO EM AÇÕES CAIXA BDR NÍVEL I,861894.2447142856
